In [1]:
import yfinance as yf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pylab import rcParams
import matplotlib.gridspec as gridspec
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import os.path, time
import datetime

In [2]:
from sklearn import preprocessing

# Keras Model and Tensorflow
import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras import optimizers
from keras.models import Sequential
from keras.utils import plot_model

In [3]:
import warnings
from IPython.display import Image
warnings.filterwarnings('ignore')
# Default parameters for plots
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['figure.titlesize'] = 16
matplotlib.rcParams['figure.figsize'] = [16, 9]
#np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.6f}'.format
# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 200)
np.set_printoptions(suppress=True)
pd.set_option('display.max_rows', None)

In [4]:
cd E:\Optimal Portfolio

E:\Optimal Portfolio


In [5]:
HSI = pickle.load(open('HSI_components.obj','rb'))
HSI = list(HSI)

In [6]:
HSI_data = yf.download(HSI,'2003-01-01','2023-06-19')

[*********************100%***********************]  73 of 73 completed


In [7]:
HSI_Index = yf.download('2800.hk', '2003-01-01','2023-06-19')

[*********************100%***********************]  1 of 1 completed


In [8]:
CSOP_Index = yf.download('7500.hk', '2003-01-01','2023-06-19')

[*********************100%***********************]  1 of 1 completed


In [9]:
CC = pickle.load(open('China_Connect.obj','rb'))
CC = CC.astype('float')
CC.tail()

,China_Connect
date,
2023-06-13,27580.080000
2023-06-14,25055.040000
2023-06-15,38200.350000
2023-06-16,38210.110000
2023-06-19,34157.540000


In [10]:
HSI_Wgt = pickle.load(open('HSI_Wgt.obj','rb'))

In [11]:
HSI_Close = HSI_data['Adj Close']
HSI_Vol = HSI_data['Volume']

In [12]:
Data = pd.merge(HSI_Close[HSI_Wgt.index],CC.astype('float'),left_index=True, right_index=True)
#Data = pd.merge(Data,CNYHKD['Adj Close'],left_index=True, right_index=True)
Data = pd.merge(Data,HSI_Index['Adj Close'],left_index=True, right_index=True)
Data = pd.merge(Data,CSOP_Index['Adj Close'],left_index=True, right_index=True)
Data.columns = HSI_Close[HSI_Wgt.index].columns.tolist() + ['CC','HSI','CSOP']

In [13]:
Data = Data.dropna()

In [14]:
HSIweeks = [week for stamp, week in Data.dropna().loc[Data.index].resample("W")]

In [15]:
HSIweeks[0]

,0005.HK,1299.HK,0700.HK,9988.HK,3690.HK,0939.HK,0941.HK,0388.HK,1398.HK,9618.HK,3988.HK,2318.HK,0883.HK,1211.HK,1810.HK,2269.HK,0016.HK,0669.HK,0002.HK,0001.HK,3968.HK,0823.HK,2331.HK,2020.HK,2388.HK,1113.HK,0011.HK,2319.HK,0386.HK,1109.HK,9633.HK,0027.HK,0981.HK,1093.HK,1088.HK,0003.HK,0066.HK,2628.HK,0857.HK,2688.HK,9999.HK,0291.HK,0688.HK,0006.HK,0267.HK,1997.HK,0175.HK,2382.HK,0992.HK,2313.HK,CC,HSI,CSOP
2020-09-08,29.451006,76.459091,501.178619,269.600006,238.000000,4.433666,41.070065,343.505463,3.526441,303.539825,2.009599,69.100746,5.897998,81.929939,22.400000,59.933334,81.831947,88.712395,65.546867,42.192860,34.340088,54.790527,30.971710,72.014214,18.890289,36.009052,109.236359,36.316101,2.500718,30.962084,31.912682,57.599556,18.799999,8.990964,9.176077,9.421366,36.199173,16.148876,1.824529,82.185883,145.327896,47.119427,17.870296,34.635178,5.274539,28.353046,15.598796,113.839127,4.324651,117.403275,25735.360000,23.054892,6.495000
2020-09-09,29.183270,76.317848,496.743408,263.200012,232.000000,4.362284,41.300148,343.132080,3.502506,294.184845,2.001810,68.803459,5.791408,78.603081,22.100000,59.533333,82.616028,89.896484,65.983261,41.892101,34.474228,55.052685,30.632956,70.959404,18.890289,36.052803,108.878792,34.865391,2.457603,30.445332,32.009098,56.956257,18.420000,8.933622,9.066512,9.657319,36.199173,15.804549,1.810219,81.146164,142.045334,46.829762,17.786982,34.676212,5.266817,28.669085,15.216940,114.817131,4.394123,117.497742,18577.530000,22.927719,6.605000
2020-09-10,28.915535,75.187904,498.714630,262.600006,233.000000,4.298832,40.763287,339.211639,3.446657,293.587708,1.978443,68.718513,5.748772,80.688568,22.450001,59.333332,82.450951,89.422844,64.892296,41.935059,34.071808,54.965302,31.020102,71.295021,18.848030,36.009052,107.716698,35.300602,2.450417,29.928579,34.612247,57.352131,18.240000,8.853037,8.956947,9.707883,35.836735,15.666820,1.781599,81.949585,143.437943,47.215984,17.703672,34.265842,5.205036,28.533642,15.083291,114.034729,4.480964,116.647652,16345.850000,22.836878,6.665000
2020-09-11,28.781662,75.658714,508.570648,266.000000,242.800003,4.275038,40.763287,344.438873,3.430700,293.587708,1.978443,68.973335,5.762983,86.547806,23.250000,61.433334,82.698547,92.312042,64.979553,41.720230,33.848240,54.921604,31.697609,72.397774,18.974815,35.790279,108.431839,35.929249,2.436045,29.885513,33.744530,59.183044,19.480000,8.876060,9.025426,9.640466,35.746117,15.684035,1.781599,81.760544,142.841141,47.602211,17.662018,34.265842,5.181869,28.398197,15.465147,115.697327,4.480964,120.520172,16454.570000,22.945885,6.565000


In [16]:
len(HSIweeks)

145

In [17]:
HSIweeks[3]

,0005.HK,1299.HK,0700.HK,9988.HK,3690.HK,0939.HK,0941.HK,0388.HK,1398.HK,9618.HK,3988.HK,2318.HK,0883.HK,1211.HK,1810.HK,2269.HK,0016.HK,0669.HK,0002.HK,0001.HK,3968.HK,0823.HK,2331.HK,2020.HK,2388.HK,1113.HK,0011.HK,2319.HK,0386.HK,1109.HK,9633.HK,0027.HK,0981.HK,1093.HK,1088.HK,0003.HK,0066.HK,2628.HK,0857.HK,2688.HK,9999.HK,0291.HK,0688.HK,0006.HK,0267.HK,1997.HK,0175.HK,2382.HK,0992.HK,2313.HK,CC,HSI,CSOP
2020-09-28,27.487602,72.504318,498.714630,263.399994,237.000000,4.021232,38.385746,336.598022,3.231241,288.611633,1.892762,68.784081,5.293986,104.175156,20.200001,60.900002,80.800278,94.111855,63.626736,40.001587,34.027096,54.659451,33.536556,74.267654,17.765707,33.383842,104.230431,34.913746,2.256395,29.251398,34.371216,51.166615,17.860001,8.507665,9.545860,9.421366,35.746117,14.840438,1.765161,80.909851,143.238998,45.517021,16.501905,33.732361,4.460640,28.037012,14.567787,115.990730,4.298598,120.709084,18439.130000,22.055668,7.075000


In [18]:
HSIweeks = [x for x in HSIweeks if len(x)>0]

In [19]:
HSIweeks_ret = []
for i in range(1,len(HSIweeks)):
    HSIweeks_ret.append((HSIweeks[i].iloc[-1] - HSIweeks[i-1].iloc[-1])/HSIweeks[i-1].iloc[-1])

In [20]:
HSIweeks_ret = pd.concat(HSIweeks_ret,axis=1).T

In [21]:
HSIweeks_ret.tail()

,0005.HK,1299.HK,0700.HK,9988.HK,3690.HK,0939.HK,0941.HK,0388.HK,1398.HK,9618.HK,3988.HK,2318.HK,0883.HK,1211.HK,1810.HK,2269.HK,0016.HK,0669.HK,0002.HK,0001.HK,3968.HK,0823.HK,2331.HK,2020.HK,2388.HK,1113.HK,0011.HK,2319.HK,0386.HK,1109.HK,9633.HK,0027.HK,0981.HK,1093.HK,1088.HK,0003.HK,0066.HK,2628.HK,0857.HK,2688.HK,9999.HK,0291.HK,0688.HK,0006.HK,0267.HK,1997.HK,0175.HK,2382.HK,0992.HK,2313.HK,CC,HSI,CSOP
136,0.011915,-0.010658,0.022456,-0.025989,-0.024279,0.003766,0.006061,-0.018088,0.020785,-0.052486,0.012461,-0.020572,0.012678,0.003289,-0.014519,-0.084691,0.002907,0.010856,-0.021577,-0.013514,-0.027848,0.003027,-0.053150,-0.038718,-0.008163,-0.008772,-0.002671,-0.015924,0.007797,-0.054492,0.042375,0.018609,0.029703,-0.028646,0.005465,-0.005348,-0.003876,0.002725,0.005587,-0.013736,0.019897,0.000000,-0.084694,0.001083,0.014000,-0.046429,-0.035824,0.003659,0.031704,-0.050445,0.016603,-0.007595,0.010526
137,-0.018503,-0.016468,-0.032413,-0.046088,-0.020218,-0.045028,-0.030121,-0.032895,-0.042986,-0.051749,-0.043077,-0.065753,-0.023474,-0.038525,-0.033149,0.018980,-0.016425,-0.024127,-0.020356,-0.011424,-0.062500,-0.023139,-0.057173,-0.036458,-0.024691,-0.005626,-0.025893,-0.022654,-0.029013,-0.054517,-0.009490,-0.050962,-0.064423,-0.030831,-0.027174,-0.014785,-0.010376,-0.066576,-0.033333,-0.077994,-0.057081,-0.061688,-0.066890,-0.043252,-0.062130,-0.003745,-0.014862,-0.054071,-0.085787,-0.030469,0.243541,-0.032143,0.071023
138,-0.002571,0.019608,0.036601,0.048951,-0.034921,-0.013753,0.000000,0.028571,-0.007092,0.064566,-0.003215,0.007820,-0.027244,0.044331,0.022857,0.034924,0.000000,-0.040338,-0.013269,-0.016512,0.022222,-0.028836,0.020948,0.012613,0.002110,0.013921,-0.020165,0.033113,-0.003984,0.044481,0.020359,0.009119,0.024666,-0.058091,-0.054004,-0.016371,-0.020397,-0.027656,0.021073,0.020818,0.045211,0.008928,0.015532,-0.003550,-0.014721,-0.005013,0.023707,-0.016458,0.050420,0.070911,0.200677,0.011597,-0.029178
139,0.030928,0.035256,0.005386,0.016970,0.045230,0.023904,0.046832,0.021164,0.033333,0.046931,0.029032,0.036857,0.042682,0.025306,0.009311,0.008999,0.008841,0.009491,0.010648,0.034627,0.028533,0.003181,0.002160,0.008304,0.029474,0.033181,0.055192,-0.022840,0.034000,0.062962,0.002347,0.016064,0.060682,0.005970,0.025591,-0.001704,0.015068,0.013473,0.082552,0.026754,0.073276,0.042281,0.041176,0.002375,0.056564,0.062972,0.026911,0.019066,0.013333,0.040784,-0.320354,0.024492,-0.050091
140,0.016667,0.012384,0.080357,0.091776,0.085759,-0.013619,-0.023791,0.026554,-0.029954,0.085517,-0.015674,0.020046,-0.053872,0.069765,0.022140,0.057971,-0.002921,0.127602,0.006146,-0.001014,-0.001321,0.003297,0.078858,0.077059,-0.006135,-0.012182,0.006206,0.061564,-0.069307,0.057276,0.012881,0.047431,0.066194,0.035608,0.026871,0.012913,-0.010796,0.022157,0.005199,0.038348,0.064252,0.027358,0.012429,-0.005924,0.016162,0.020142,0.041929,0.060645,0.122368,0.157394,0.485797,0.034588,-0.066155


In [22]:
""" Here slices for windowing are created """
l = 30
m = 0
n = 1

np_data = np.array(HSIweeks_ret)  # Convert to the numpy array for slicing data
N = np_data.shape[0] # Total number of samples
k = N - (l + m + n)

# Create Input and output Slice
in_slice = np.array([range(i, i + l) for i in range(k)])
op_slice = np.array([range(i + l + m, i + l + m + n) for i in range(k)])

In [30]:
op_slice[1],in_slice[1]

(array([31]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]))

In [23]:
len(in_slice)

110

In [31]:
np_data.shape

(141, 53)

In [82]:
def LSTM_model(inp1,inp2,p,n,X,Y):
    model = Sequential()
    # Defining hidden layer number and the shape of the input (number of data in the dataset and the number of feature)
    model.add(LSTM(p, input_shape=(inp1, inp2)))
    # Add forget (dropout) layer with probability per argument
    model.add(Dropout(0.2))
    # End the network with hiddenlayer per the size of forecast day e.g. 1,5,10
    model.add(Dense(n))
    # Build and return the model per the selected optimizer
    model.compile(loss='mean_squared_error', optimizer='SGD', metrics=['accuracy'])
    model.fit(x=X, y=Y, batch_size=30, epochs=100, shuffle=True, validation_split=0.1, verbose=0)
    return model

In [126]:
pred_HSI_ret = []
for t in range(50,len(in_slice)):
    i = t-50
    X = []
    for k in range(i,t):
        X.append(np_data[in_slice[k],:])
    X = np.array(X)
    model = LSTM_model(l,np_data.shape[1],64,1,X,np_data[op_slice[i:t],-2])
    pred_HSI_ret.append((np_data[op_slice[t],-2][0],model.predict(np.array([np_data[in_slice[t],:]]))[0][0]))

1/1 [==============================] - 0s 251ms/step


In [127]:
pred_HSI_ret=pd.DataFrame(pred_HSI_ret,columns=['Actual','Predicted'])
pred_HSI_ret.tail()

,Actual,Predicted
55,-0.020337,-0.012026
56,-0.007595,-0.032540
57,-0.032143,0.003078
58,0.011597,0.033557
59,0.024492,0.014655


In [128]:
pred_HSI_ret['Correct_Direction'] = np.where(pred_HSI_ret['Actual']*pred_HSI_ret['Predicted']>0,1,0)

In [129]:
pred_HSI_ret['Correct_Direction'].sum()/len(pred_HSI_ret)

0.5666666666666667